---
## **Trainer**

خودش اتومات انجام میده Trainer کد زیر پیاده سازی دستی همون چیزی هست که کلاس

(Trainer in HuggingFace یا PyTorch Lightning)
    


In [ ]:
  for step in range(cfg.steps):
    # lr
    lr = lr_schedule(step)
    for pg in optim.param_groups:
      pg['lr'] = lr

    x,y = get_batch('train')
    with torch.cuda.amp.autocast(enabled=(device.type=='cuda')):
      logits, loss = model(x,y)
      loss = loss / cfg.accum_steps
    scaler.scale(loss).backward()

    if (step + 1) % cfg.accum_steps == 0:
      scaler.step(optim)
      scaler.update()
      optim.zero_grad(set_to_none = True)

    running_loss += loss.item() * cfg.accum_steps

    # Validation & logging every 100 steps
    if(step + 1) % 100 == 0 or (step + 1) == cfg.steps:
      model.eval()
      with torch.no_grad():
        vx,vy = get_batch('val')
        _,vloss = model(vx,vy)
      avg_train = running_loss / 100
      running_loss = 0.0
      print(f"step {step+1}/{cfg.steps} | lr {lr:2e} | train_loss {avg_train:.4f} | val_loss {vloss.item():.4f}")
      model.train()


##### خلاصه اعمال انجام شده در کد بالا :    

In [ ]:
  for step in range(cfg.steps):
      1. lr تنظیم
      2. batch گرفتن
      3. forward + loss
      4. backward (GradScaler با )
      5. update optimizer (accum_steps هر)
      6. validation & logging (step 100 هر)

---
## تکنیک های مفید آموزش :      

1. AMP (Automatic Mixed Precision)
2. Scaler
3. Gradient Accumulation


فرآیند آموزش مدل های عمیق, به ویژه مدل های بزرگ مانند ترنسفورمرها,میتواند شامل تکنیک های پیشرفته ای باشد که برای بهینه سازی عملکرد,سرعت و کارایی استفاده میشوند.تکنیک های بالا از جمله این روش ها هستند

### **AMP** **(Automatic Mixed Precision)**


Mixed Precision -> دقت مختلط

تسریع آموزش مدل و کاهش مصرف حافظه با استفاده از دقت مختلط

در آموزش معمولی مدل ها, محاسبات با دقت 32 بیتی انجام میشود.این دقت بالا باعث میشود محاسبات کندتر و مصرف حافظه بیشتر باشد


از دقت 16 بیتی برای برخی محاسبات استفاده میکند,در حالی که محاسبات حساس همچنان با دقت 32 بیتی انجام میشودAMP

مثال در پایتورچ :

In [ ]:
for torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()
for data, target in dataloader:
  optimizer.zero_grad()
  with autocast():
    output = model(data)
    loss = criterion(output,target)
  scaler.scale(loss).backward()
  scaler.step(optimizer)
  scaler.update()

scaler ->  مدیریت مقیاس گرادیان ها در آموزش با دقت مختلط  

### **Gradient Accumulation**

آموزش مدل با بچ سایز های بزرگتر زمانی که حافظه محدود است

گرادیان ها روی چندین بچ جمع میشوند و سپس بعد از تعداد مشخصی یچ, پارامتر ها به روز میشوند

In [ ]:
accumulation_steps = 4
for i, (data, target) in enumerate(dataloader):
  output = model(data)
  loss = criterion(output, target)
  loss = loss / accumulation_steps
  loss.backward()
  if (i + 1) % accumulation_steps == 0:
    optimizer.step()
    optimizer.zero_grad()

### ▶ مقایسه روش قدیمی و تکنیک های بالا

#### دنیای قدیم (پروژه‌های ساده)

In [ ]:
# آموزش پایه - همون چیزی که احتمالاً دیدی
for epoch in range(epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

#### دنیای جدید (پروژه‌های حرفه‌ای)

In [ ]:
# آموزش پیشرفته - با همه تکنیک‌ها
scaler = GradScaler()
accumulation_steps = 4

for epoch in range(epochs):
    for i, batch in enumerate(dataloader):
        with autocast():
            outputs = model(batch)
            loss = criterion(outputs, labels) / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()